In [39]:
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup

### Extracting tables from the Wikipedia page

In [40]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population').text
soup = BeautifulSoup(website_url,'lxml')
My_table = soup.find('table',{'class':'wikitable sortable'})

In [41]:
# This string is used to add wikipedia.org to all the Wikipedia paes of the cities obtained from the table

wiki = 'https://en.wikipedia.org'

In [42]:
# Storing links of Wikipedia pages from the table in a list

link_list = []
for row in My_table.tbody.findAll('tr'):
    links = row.find_all('a')[0]
    #print(links['href'])
    link = wiki+links['href']
    link_list.append(link)
    #print(link)

In [43]:
# Obtains each row of tthe table

table_rows = My_table.find_all('tr')
res = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        res.append(row)


In [44]:
data = pd.DataFrame(res)
data.columns = ['2017 rank', 'City', 'State', '2017 estimate', '2010 Census',
                                                         'Change', '2016 land area Sq Mi','2016 land area Sq Km',
                 '2016 population density/sq Mi', '2016 population/sq Km', 'Location']

In [45]:
# Removing the citations (Marked  with []) next to names of cities in the table

for i, row in data.iterrows():
    row['City'] = re.sub(r'\[.*?\]', '', row['City'])
    row['2016 land area Sq Mi'] = re.sub("\D", "", row['2016 land area Sq Mi'])
    row['2016 land area Sq Km'] = re.sub("\D", "", row['2016 land area Sq Km'])
    row['2016 population density/sq Mi'] = re.sub("\D", "", row['2016 population density/sq Mi'])
    row['2016 population/sq Km'] = re.sub("\D", "", row['2016 population/sq Km'])

In [46]:
data['Wikipedia Link'] = link_list[1:]

## This segment finds the links of city municipal websites given in the Wikipedia pages of respective cities

### We visit the Wikipedia page of each city, find the table in which the website is given and store it

In [47]:
web = []
for l in link_list[1:]:
    website_url = requests.get(l).text
    soup = BeautifulSoup(website_url,'lxml')
    tables = soup.find_all('table')
    k = tables[0].findAll('a')
    j = 0
    while(len(k) < 15):
        k = tables[j].findAll('a')
        j+=1
    w = k[len(k) - 1]['href']
    if "wikidata" in w:
        w = k[len(k) - 2]['href']
    
    ### When we obtain a website, we have to make sure that it is the official page
    ### Hence, we use these stop-words which are commonly used in wikipedia pages to filter them out
    
    stop = ["city-data","pdf", "lion1906", "geonames"]
    
    
    i = 1
    while "http" not in w:
        i+=1
        w = k[len(k) - i]['href']
        #print(w)
        

    for word in stop:
        if word in w:
            i+= 1
            w = k[len(k) - i]['href']

            
## Renton, Washington is the only page that has a  link in a different table
    
    
    if 'Renton,_Washington' in l:
        k = tables[4].findAll('a')
        w = k[len(k) - 1]['href']
        
    web.append(w)

In [48]:
len(web)

314

In [49]:
data['City Website'] = web

In [50]:
data.head(5)

,2017 rank,City,State,2017 estimate,2010 Census,Change,2016 land area Sq Mi,2016 land area Sq Km,2016 population density/sq Mi,2016 population/sq Km,Location,Wikipedia Link,City Website
0,1,New York City,New York,"8,398,748","8,175,133",+2.74%,3015,78092,28317,109332,40°39′49″N 73°56′19″W﻿ / ﻿40.6635°N 73.9387°W﻿...,https://en.wikipedia.org/wiki/New_York_City,https://www.nyc.gov/
1,2,Los Angeles,California,"3,990,456","3,792,621",+5.22%,4687,121392,8484,32762,34°01′10″N 118°24′39″W﻿ / ﻿34.0194°N 118.4108°...,https://en.wikipedia.org/wiki/Los_Angeles,https://www.lacity.org/
2,3,Chicago,Illinois,"2,705,994","2,695,598",+0.39%,2273,58872,11900,46002,41°50′15″N 87°40′54″W﻿ / ﻿41.8376°N 87.6818°W﻿...,https://en.wikipedia.org/wiki/Chicago,http://www.cityofchicago.org
3,4,Houston,Texas,"2,325,502","2,100,263",+10.72%,6375,165112,3613,13952,29°47′12″N 95°23′27″W﻿ / ﻿29.7866°N 95.3909°W﻿...,https://en.wikipedia.org/wiki/Houston,http://www.houstontx.gov/
4,5,Phoenix,Arizona,"1,660,272","1,445,632",+14.85%,5176,134062,3120,12002,33°34′20″N 112°05′24″W﻿ / ﻿33.5722°N 112.0901°...,"https://en.wikipedia.org/wiki/Phoenix,_Arizona",http://www.phoenix.gov


In [51]:
data.to_csv('my_data.csv', index = False)